### modules

In [ ]:
if globals().get('LOADED_ANNOTATIONS') != True:
    %run ANNOTATIONS.ipynb

In [ ]:
if globals().get('LOADED_PATTERN_OBSERVER') == None:
    %run PATTERN_OBSERVER.ipynb

In [ ]:
if globals().get('LOADED_CHECKBOXES') == None:
    %run CHECKBOXES.ipynb

In [ ]:
if globals().get('LOADED_DATA_CONTAINER') == None:
    %run DATA_CONTAINER.ipynb

In [ ]:
if globals().get('LOADED_SETTINGS') == None:
    %run SETTINGS.ipynb

In [ ]:
if globals().get('LOADED_VIEWING_TABS_EXTRADAY_ANALYSIS') == None:
    %run VIEWING_TABS_EXTRADAY_ANALYSIS.ipynb

In [ ]:
if globals().get('LOADED_VIEWING_TABS_INTRADAY_ANALYSIS') == None:
    %run VIEWING_TABS_INTRADAY_ANALYSIS.ipynb

In [ ]:
if globals().get('LOADED_VIEWING_TABS_FACTOR_ANALYSIS') == None:
    %run VIEWING_TABS_FACTOR_ANALYSIS.ipynb

### begin

In [ ]:
class BACKTEST_VIEWER(IObserver,ISubject):
    def __init__(self,
        name='BACKTEST_VIEWER'
        ):
        self.name                 = name
        self._observers           = [] # for observer pattern
        
        ####################################
        # instantiate SETTINGS
        ####################################
        self._SETTINGS = SETTINGS(
            name                  = 'SETTINGS',
            width                 = '20%',
        )

        ####################################
        # instantiate BACKTEST_SELECTOR
        ####################################
        self._BACKTEST_SELECTOR = CHECKBOXES(
            name                  = 'BACKTEST_SELECTOR',
            options               = self._settings_all_backtests,
            default               = False,
            width                 = '60%',
        )
            
        ####################################
        # instantiate BOOK_SELECTOR
        ####################################
        self._BOOK_SELECTOR = CHECKBOXES(
            name                  = 'BOOK_SELECTOR',
            options               = self._settings_all_books, # TODO `BACKTEST SELECTOR` needs to have `all_books`
            default               = True,
            width                 = '20%',
        )

        ####################################
        # instantiate DATA_CONTAINER
        ####################################
        self._DATA_CONTAINER = DATA_CONTAINER( # TODO needs to observe changes in `self.selected_backtests`, `self.selected_books` and `self.applied_settings`
            applied_backtest_path = self._settings_backtest_path, # RENAME applied_* to *
            applied_view_type     = self._settings_view_type,
            applied_plot_type     = self._settings_plot_type,
            applied_return_type   = self._settings_return_type,
            applied_date_from     = self._settings_date_from,
            applied_date_to       = self._settings_date_to,
            selected_backtests    = self._selected_backtests,
            selected_books        = self._selected_books,
        )
        
        ####################################
        # build GUI
        ####################################
        self.reference            = {}
        self.widget               = self._make_widget(self._settings_view_type) # TODO needs to observe changes in self._SETTINGS

        ###########################
        # class linking
        ###########################
        # BACKTEST_VIEWER observes ...
        self._SETTINGS.attach(self)                           # `applied_view_type` from SETTINGS
        
        # BACKTEST_SELECTOR observes ...
        self._SETTINGS.attach(self._BACKTEST_SELECTOR)        # `all_backtests` from SETTINGS

        # BOOK_SELECTOR observes ...
        #self._BACKTEST_SELECTOR.attach(self._BOOK_SELECTOR)  # `all_backtests` from `BACKTEST SELECTOR` # TODO

        # DATA_CONTAINER observes ...                         # (must come after BACKTEST_SELECTOR and BOOK_SELECTOR)
        self._SETTINGS.attach(self._DATA_CONTAINER)           # `backtest_path` from SETTINGS, i.e. backtest path, etc
        self._BACKTEST_SELECTOR.attach(self._DATA_CONTAINER)  # `applied_settings` from BACKTEST_SELECTOR, i.e. changes in selected backtests
        self._BOOK_SELECTOR.attach(self._DATA_CONTAINER)      # `applied_settings` from BOOK_SELECTOR, i.e. changes in selected books
        self.attach(self._DATA_CONTAINER)                     # BACKTEST_VIEWER force TAB_COLLECTION to replot, it does so via DATA_CONTAINER

        # VIEWING_TAB observes ...
        self._DATA_CONTAINER.attach(self._TAB_COLLECTION)     # `df` from DATA_CONTAINER
        #self._BOOK_SELECTOR.attach(self._TAB_COLLECTION)     # `DATA` from DATA_CONTAINER

    ####################################
    # observer pattern
    ####################################
    # subject
    def attach(self,observer : IObserver) -> None :
        print('OBSERVER PATTERN',':',observer.name,'OBSERVES',self.name)
        self._observers.append(observer)
        
    def detach(self,observer : IObserver) -> None :
        print('OBSERVER PATTERN',':',observer.name,'STOPS OBSERVING',self.name)
        self._observers.remove(observer)
        
    def notify(self,info) -> None :
        print('OBSERVER PATTERN',':',self.name,'NOTIFIES',len(self._observers),'OBSERVERS')
        
        # print receivers
        for observer in self._observers:
            print('OBSERVER PATTERN',':',self.name,'NOTIFIES',observer.name)
            observer.react(self.name,info)
            
    def react(self,
        subject_name : str,
        subject_info : object
        ) -> None :
        print('OBSERVER PATTERN',':',self.name,'REACTS','subject_name',subject_name)
        print('OBSERVER PATTERN',':',self.name,'REACTS','subject_info',subject_info.keys())

        # SETTINGS changed
        if subject_name == 'SETTINGS':
            if 'reload_data' in subject_info:
                print(self.name,':','pass','reload_data nothing to do with me')
            else:
                # update viewing tabs as needed
                if self.reference['applied_view_type'] != subject_info['view_type']:
                    print(self.name,':','view_type',subject_info['view_type'])

                    # change backend
                    self._DATA_CONTAINER.detach(self._TAB_COLLECTION) # deregister existing tabs
                    self._TAB_COLLECTION = self._make_tab_collection(subject_info['view_type']) # make new tab collection
                    self._DATA_CONTAINER.attach(self._TAB_COLLECTION) # register new tabs

                    # update frontend
                    self._tab_widgets = self._TAB_COLLECTION.widget.children
                    
                    # force replot
                    self._notify()
                else:
                    print(self.name,':','pass','view_type did not change')
                        
    ####################################
    # build GUI
    ####################################
    def _make_widget(self,view_type : str) -> VBox :
        # controls
        controls = HBox(
            [
                self._BACKTEST_SELECTOR.widget,
                self._BOOK_SELECTOR.widget,
                self._SETTINGS.widget,
            ]
        )
        
        # make viewing tabs
        self._TAB_COLLECTION = self._make_tab_collection(view_type)

        # build screen
        return VBox(
            [
                controls,
                self._TAB_COLLECTION.widget
            ]
        )
    
    def _make_tab_collection(self,view_type : str) -> TAB_COLLECTION :
        # update reference
        self.reference['applied_view_type'] = view_type
        
        # instantiate
        function_name = TAB_COLLECTION._function_dico[view_type]
        
        # debugging
        print(self.name,':','_make_tab_collection',view_type,function_name)
        
        # make and return
        return eval(function_name)

    ####################################
    # properties
    ####################################
    # from self._SETTINGS
    @property
    def _settings_backtest_path(self) -> str :
        return self._SETTINGS.reference['backtest_path']

    @property
    def _settings_view_type(self) -> str :
        return self._SETTINGS.reference['view_type']

    @property
    def _settings_plot_type(self) -> str :
        return self._SETTINGS.reference['plot_type']

    @property
    def _settings_return_type(self) -> str :
        return self._SETTINGS.reference['return_type']

    @property
    def _settings_date_from(self) -> object :
        return self._SETTINGS.reference['date_from']

    @property
    def _settings_date_to(self) -> object :
        return self._SETTINGS.reference['date_to']

    @property
    def _settings_all_backtests(self) -> ID_PATHS :
        return self._SETTINGS.reference['all_backtests']

    @property
    def _settings_all_books(self) -> ID_PATHS :
        return self._SETTINGS.reference['all_books']

    # from self._BACKTEST_SELECTOR
    @property
    def _selected_backtests(self) -> OPTIONID_VALUES:
        return self._BACKTEST_SELECTOR.applied_settings
        
    # from self._BOOK_SELECTOR
    @property
    def _selected_books(self) -> OPTIONID_VALUES:
        return self._BOOK_SELECTOR.applied_settings

    # from self.widget
    @property
    def _control_widgets(self) -> BUTTONS:
        return self.widget.children[0].children
    
    @property
    def _tab_widgets(self) -> TAB_COLLECTION:
        return self.widget.children[1].children

    @_tab_widgets.setter
    def _tab_widgets(self,new_tab_collection : TAB_COLLECTION) -> None:
        self.widget.children[1].children = new_tab_collection

In [ ]:
# example
a=BACKTEST_VIEWER()

In [ ]:
dir(a)

In [ ]:
a.widget

backtest
	- no ID, take path from settings

days of the week plot

check force reload

settings[reload] = double plot

settings
	notify
		shouldnt cause double plotting